In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
df = pd.read_csv('train_E6oV3lV.csv')
df['label'] = df['label'].map({0: 2, 1: 1})
df = df.drop('id', axis=1)
df.head()

,label,tweet
0,2,@user when a father is dysfunctional and is s...
1,2,@user @user thanks for #lyft credit i can't us...
2,2,bihday your majesty
3,2,#model i love u take with u all the time in ...
4,2,factsguide: society now #motivation


In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import copy

In [4]:
train, test = train_test_split(df, test_size=0.2)

In [5]:
train

,label,tweet
27747,1,@user just got done threatening my coworker to...
13529,2,happy #caturday !! #catstagram #catlife @ so...
30762,2,sometimes i feel that i have given too much ri...
26521,2,my laptop's broken now cant binge watch crimin...
11939,2,#model i love u take with u all the time in ...
...,...,...
20710,2,@user feeling #proud and to see bold #vision...
27138,2,how to be perfectly alone: 9 transformationa...
20701,2,#london attack bull chase: when you leave th...
4965,2,i finally found a way how to delete old tweets...


In [49]:
train_df = copy.deepcopy(train)
test_df = copy.deepcopy(test)

train_texts = train_df['tweet'].values
train_texts = [s.lower() for s in train_texts]

test_texts = test_df['tweet'].values
test_texts = [s.lower() for s in test_texts]

The word_index assigns a unique index to each word present in the text. This unique integer helps the model during training purposes.

In [50]:
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(train_texts)

alphabet = "abcdefghijklmnopqrstuvwxyz0123456789 ,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"

char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1
tk.word_index = char_dict.copy()
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1

#texts_to_sequences method helps in converting tokens of text corpus into a sequence of integers.
train_sequences = tk.texts_to_sequences(train_texts)
test_texts = tk.texts_to_sequences(test_texts)

#pad_sequences is used to ensure that all sequences in a list have the same length.
train_data = pad_sequences(train_sequences, maxlen=150, padding='post')
test_data = pad_sequences(test_texts, maxlen=150, padding='post')

train_data = np.array(train_data, dtype='float32')
test_data = np.array(test_data, dtype='float32')

# train_classes = train_df['label'].values
# train_class_list = [x - 1 for x in train_classes]
# test_classes = test_df['label'].values
# test_class_list = [x - 1 for x in test_classes]

# from keras.utils import to_categorical

# train_classes = to_categorical(train_class_list)
# test_classes = to_categorical(test_class_list)

In [151]:
train_classes = train_df['label'].values
train_class_list = [x - 1 for x in train_classes]

test_classes = test_df['label'].values
test_class_list = [x - 1 for x in test_classes]

from keras.utils import to_categorical
#one hot encode integer data
train_classes = to_categorical(train_class_list)
test_classes = to_categorical(test_class_list)

ab = np.argmax(train_classes, axis = 1)
ab

array([0, 1, 1, ..., 1, 1, 1])

In [150]:
train_classes

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [64]:
vocab_size = len(tk.word_index)
#unique integers assigned to each word

70

In [11]:
embedding_weights = [] #(71, 70)
embedding_weights.append(np.zeros(vocab_size)) # first row is pad

for char, i in tk.word_index.items(): # from index 1 to 70
    onehot = np.zeros(vocab_size)
    onehot[i-1] = 1
    embedding_weights.append(onehot)
embedding_weights = np.array(embedding_weights)

In [12]:
embedding_weights = [] #(71, 70)
embedding_weights.append(np.zeros(vocab_size)) # first row is pad

for char, i in tk.word_index.items(): # from index 1 to 70
    onehot = np.zeros(vocab_size)
    onehot[i-1] = 1
    embedding_weights.append(onehot)
embedding_weights = np.array(embedding_weights)

In [13]:
from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model

In [14]:
input_size = 150
embedding_size = 70
conv_layers = [[256, 7, 3], 
               [256, 7, 3], 
               [256, 3, -1], 
               [256, 3, -1], 
               [256, 3, -1], 
   
               [256, 3, 3]]

fully_connected_layers = [1024, 1024]
num_of_classes = 2
dropout_p = 0.5
optimizer = 'adam'
loss = 'categorical_crossentropy'

In [15]:
embedding_layer = Embedding(vocab_size+1, 
                            embedding_size,
                            input_length=input_size,
                            weights=[embedding_weights])

In [16]:
inputs = Input(shape=(input_size,), name='input', dtype='int64')  
x = embedding_layer(inputs)
for filter_num, filter_size, pooling_size in conv_layers:
    x = Conv1D(filter_num, filter_size)(x) 
    x = Activation('relu')(x)
    if pooling_size != -1:
        x = MaxPooling1D(pool_size=pooling_size)(x) 
x = Flatten()(x) 

for dense_size in fully_connected_layers:
    x = Dense(dense_size, activation='relu')(x) 
    x = Dropout(dropout_p)(x)

predictions = Dense(num_of_classes, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy']) 
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 150)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 150, 70)           4970      
_________________________________________________________________
conv1d (Conv1D)              (None, 144, 256)          125696    
_________________________________________________________________
activation (Activation)      (None, 144, 256)          0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 48, 256)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 42, 256)           459008    
_________________________________________________________________
activation_1 (Activation)    (None, 42, 256)           0     

In [72]:
model.fit(train_data, train_classes,  epochs=20, batch_size=30, verbose=1)

Epoch 1/20
853/853 [==============================] - 227s 266ms/step - loss: 0.0580 - accuracy: 0.9815
Epoch 2/20
853/853 [==============================] - 220s 258ms/step - loss: 0.0547 - accuracy: 0.9823
Epoch 3/20
853/853 [==============================] - 220s 258ms/step - loss: 0.0392 - accuracy: 0.9881
Epoch 4/20
853/853 [==============================] - 221s 259ms/step - loss: 0.0363 - accuracy: 0.9883
Epoch 5/20
853/853 [==============================] - 221s 259ms/step - loss: 0.0332 - accuracy: 0.9894
Epoch 6/20
853/853 [==============================] - 223s 261ms/step - loss: 0.0331 - accuracy: 0.9902
Epoch 7/20
853/853 [==============================] - 227s 266ms/step - loss: 0.0281 - accuracy: 0.9916
Epoch 8/20
853/853 [==============================] - 226s 265ms/step - loss: 0.0232 - accuracy: 0.9926
Epoch 9/20
853/853 [==============================] - 231s 271ms/step - loss: 0.0191 - accuracy: 0.9947
Epoch 10/20
853/853 [==============================] - 220s 258m

In [79]:
test_loss, test_acc = model.evaluate(test_data,  test_classes)

200/200 [==============================] - 12s 58ms/step - loss: 0.4175 - accuracy: 0.9676
